In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
ms = MinMaxScaler(feature_range=(0,1))
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [ ]:
import numpy as np
import pandas as pd
dat = pd.read_csv('/content/drive/MyDrive/Deep Learning/sample demand forecast.csv')
dat

In [ ]:
dat_train = pd.read_csv('/content/drive/MyDrive/Deep Learning/train.csv')
df1 = pd.DataFrame(dat_train)
df1 = df1.drop(['date'],axis=1)
df1

In [ ]:
sc.fit(np.reshape(np.array(df1['sales']),(-1,1)))
df1['sales'] = sc.fit_transform(np.reshape(np.array(df1['sales']),(-1,1)))
df1


In [ ]:
x=1
y=1
while(x!=11):
  y=1
  while(y!=51):
    op = df1.loc[(df1['item']==y) & (df1['store']==x)]
    print(op)
    y=y+1
  x=x+1

In [ ]:
X = []
time = 60
i=1
while(i!=11):
  a = []
  j=1
  while(j!=51):
    k=0
    b = []
    op = np.array(df1.loc[(df1['item']==j) & (df1['store']==i)])
    len = np.shape(op)[0]
    while(k!=len-time):
      c = []
      l=0
      while(l!=time):
        c.append(op[k+l][2])
        l=l+1
      b.append(c)
      k=k+1
    a.append(b)
    j=j+1
  X.append(a)
  i=i+1

np.shape(X)


In [ ]:
np.shape(np.array(X[0][0]))[0]

In [ ]:
y = []
time = 60
for i in range(1,11):
  a = []
  for j in range(1,51):
    b = []
    op = np.array(df1.loc[(df1['item']==j) & (df1['store']==i)])
    len = np.shape(op)[0]
    for k in range(0,len-time):
      b.append(op[k+time][2])
    a.append(b)
  y.append(a)

np.shape(y)

In [ ]:
print(np.array(y[0][0]))

In [ ]:
dat_test = pd.read_csv('/content/drive/MyDrive/Deep Learning/test.csv')
dat_test

In [ ]:
model=Sequential()

model.add(LSTM(units = 50, return_sequences = True, input_shape = (60,1)))
model.add(Dropout(rate = 0.2))

model.add(LSTM(units = 50, return_sequences = False))
model.add(Dropout(rate = 0.2))

model.add(Dense(25))
model.add(Dense(1))

model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
y_pred = []
for i in range(0,20):
  p = dat_test['store'][i]
  q = dat_test['item'][i]
  X_test = []
  X = np.array(X)
  dom = 1766
  for j in range(1,60):
    X_test.append(X[p-1][q-1][dom-1][j])
  X_test.append(y[p-1][q-1][dom-1])
  temp = np.zeros((1,60,1))
  for j in range(0,60):
    temp[0][j][0] = X_test[j]
  
  model.fit(np.array(X[p-1][q-1]),np.array(y[p-1][q-1]),epochs=1,batch_size=20,verbose=1)
  pred = model.predict(temp)
  pred = sc.inverse_transform(pred)
  y_pred.append(pred)


In [ ]:
y_pred

In [ ]:
k=np.zeros(np.shape(y_pred)[0])
for i in range(0,np.shape(y_pred)[0]):
  k[i]=y_pred[i][0][0]
k

In [ ]:
t = dat_train['sales'][0:20]
t

In [ ]:
s=0
for i in range(0,20):
  s = s+(t[i]-k[i])**2
np.sqrt(s)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(t)
plt.plot(k)